# Searching for High PM Stars in Other Catalogs

In this notebook, we seek to explore the appearance of high PM stars in various catalogs, obtained from https://data.lsdb.io/. This will be useful in gauging wether our current method will be capable of identifying these high PM stars, and wether other techniques are viable (i.e. KBMOD). We will search for these stars by first utilizing PM data from Gaia DR3, and crossmatching Gaia stars with high proper motions with those from the other catalogs 

In [2]:
from pathlib import Path

import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt

from dask.distributed import Client
import dask.array
from dask.dataframe.utils import make_meta

from hats import read_hats
from hats.inspection import plot_pixels
from hats_import.catalog.file_readers import CsvReader
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import ImportArguments, pipeline_with_client

import lsdb

from catalog_filtering import bandFilterLenient, contains_PM

print("Imported libraries.")

Imported libraries.


In [10]:
pm_speed_min = 2000 #units are milliarcseconds per year
pm_speed_max = 10**5
CATALOG_DIR = Path("../../catalogs")
GAIA_2000_NAME = 'gaia_dr3_pm_greater_2000'
GAIA_2000_DIR = CATALOG_DIR / GAIA_2000_NAME

gaia_pm_greater_100 = lsdb.read_hats('../../catalogs/gaia_dr3_pm_greater_100', margin_cache='../../catalogs/margin_caches/gaia_margin_cache_18_arcsec')
print(len(gaia_pm_greater_100))
gaia_pm_greater_100

506589


,source_id,ra,dec,pmra,pmdec
npartitions=3917,,,,,
"Order: 2, Pixel: 0",int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 3, Pixel: 4",...,...,...,...,...
...,...,...,...,...,...
"Order: 4, Pixel: 3067",...,...,...,...,...
"Order: 3, Pixel: 767",...,...,...,...,...


In [11]:
%%time
with Client():
    pm_expr = f'{pm_speed_max**2} > (pmra**2 + pmdec**2) > {pm_speed_min**2}'
    gaia_pm_greater_100.query(pm_expr).to_hats(
        catalog_name=GAIA_2000_NAME, 
        base_catalog_path=GAIA_2000_DIR
    )

CPU times: user 4min 20s, sys: 3.69 s, total: 4min 23s
Wall time: 5min 28s


In [15]:
gaia_pm_greater_2000 = lsdb.read_hats(GAIA_2000_DIR).compute()
print(len(gaia_pm_greater_2000))
gaia_pm_greater_2000

87


,source_id,ra,dec,pmra,pmdec
_healpix_29,,,,,
9282739332438892,18565464288396416,39.071677,6.87809,1801.671458,1450.486947
9658118652315295,19316224572460416,39.028385,6.893339,1778.585494,1477.306159
...,...,...,...,...,...
3441196866318773055,6882393725554653056,316.218165,-16.96794,-916.062823,-2039.790599
3442888062150387257,6885776098199761024,317.325879,-13.311372,715.064154,-1995.445621


## Crossmatching Against ZTF-DR14 (objects)

In [13]:
ztf_14_obj = (
    lsdb.read_hats('https://data.lsdb.io/hats/ztf_dr14/ztf_object')
)
ztf_14_obj

,ps1_objid,ra,dec,ps1_gMeanPSFMag,ps1_rMeanPSFMag,ps1_iMeanPSFMag,nobs_g,nobs_r,nobs_i,mean_mag_g,mean_mag_r,mean_mag_i
npartitions=2352,,,,,,,,,,,,
"Order: 3, Pixel: 0",int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],int32[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 3, Pixel: 1",...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 4, Pixel: 3070",...,...,...,...,...,...,...,...,...,...,...,...
"Order: 4, Pixel: 3071",...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
# TODO: Perform Conesearch in ZTF of some size, ensure that it is not empty, and plot to observe HPMS behavior. 
# Also, ask Kostya regarding alternative to doing this.
#Finding conesearch radius, use https://irsa.ipac.caltech.edu/data/ZTF/docs/releases/dr14/ztf_release_notes_dr14.pdf and search 
#gaia dr3 observing time into google

In [ ]:
obj_1 = ztf_14_obj.cone_search(ra=gaia_pm_greater_2000.iloc[[0]]['ra'], 
                        dec=gaia_pm_greater_2000.iloc[[0]]['dec'], 
                        radius_arcsec=)